# Otosclerosis analysis 

## Variables for Columbia's cluster

In [ ]:
# Common variables Columbia's cluster
UKBB_PATH=$HOME/UKBiobank
UKBB_yale=$HOME/UKBiobank_Yale_transfer
USER_PATH=$HOME/project
container_lmm=$HOME/containers/lmm.sif
container_marp=$HOME/containers/marp.sif
container_annovar=$HOME/containers/gatk4-annovar.sif
hearing_pheno_path=$UKBB_PATH/phenotype_files/
tpl_file=$USER_PATH/bioworkflows/admin/csg.yml
formatFile_fastgwa=$USER_PATH/UKBB_GWAS_dev/data/fastGWA_template.yml
formatFile_bolt=$USER_PATH/UKBB_GWAS_dev/data/boltlmm_template.yml
formatFile_saige=$USER_PATH/UKBB_GWAS_dev/data/saige_template.yml
formatFile_regenie=$USER_PATH/UKBB_GWAS_dev/data/regenie_template.yml

In [3]:
# Workflows
lmm_sos=$USER_PATH/bioworkflows/GWAS/LMM.ipynb
anno_sos=$USER_PATH/bioworkflows/variant-annotation/annovar.ipynb
clumping_sos=$USER_PATH/bioworkflows/GWAS/LD_Clumping.ipynb
extract_sos=$USER_PATH/bioworkflows/GWAS/Region_Extraction.ipynb
snptogene_sos=$USER_PATH/UKBB_GWAS_dev/workflow/snptogene.ipynb

# LMM directories for imputed data
lmm_imp_dir_fastgwa=$UKBB_PATH/results/FastGWA_results/results_imputed_data
lmm_imp_dir_bolt=$UKBB_PATH/results/BOLTLMM_results/results_imputed_data
lmm_imp_dir_saige=$UKBB_PATH/results/SAIGE_results/results_imputed_data
lmm_imp_dir_regenie=$UKBB_PATH/results/REGENIE_results/results_imputed_data

# LMM directories for exome data
lmm_exome_dir_fastgwa=$UKBB_PATH/results/FastGWA_results/results_exome_data
lmm_exome_dir_bolt=$UKBB_PATH/results/BOLTLMM_results/results_exome_data
lmm_exome_dir_saige=$UKBB_PATH/results/SAIGE_results/results_exome_data
lmm_exome_dir_regenie=$UKBB_PATH/results/REGENIE_results/results_exome_data

In [9]:
## LMM variables 
### Specific to REGENIE
bsize=1000
lowmem=$HOME/scratch60/
lowmem_dir=$HOME/scratch60/predictions
trait=bt
minMAC=4
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
reverse_log_p=True
numThreads=4
lmm_job_size=1
ylim=0

## Single variant association analysis for otosclerosis

In [10]:
lmm_dir_regenie=$lmm_exome_dir_regenie/070722_otosclerosis_200K
lmm_sbatch_regenie=$lmm_exome_dir_regenie/070722_otosclerosis_200K/otosclerosis_200K_exome_regenie_$(date +"%Y-%m-%d").sbatch
phenoFile=$hearing_pheno_path/HI_UKBB/otosclerosis/070122_oto_200k.382case_97050ctrl.tsv
phenoCol=oto
covarCol=sex
qCovarCol="age"
bgenMinINFO=0.8
bgenMinMAF=0.001
minMAC=4
#Use the original bed files that passed QC using Megan's parameters geno=0.01, mind=0.1, maf=0.01, hwe=5e-08
bfile=$UKBB_PATH/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
# Use the QC'ed exome files variant and sample missingness < 10%
genoFile=`echo $UKBB_PATH/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
label_annotate=SNP

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_dir $lowmem_dir
    --trait $trait 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --ylim $ylim
    --label_annotate $label_annotate
    --no-annotate
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/UKBiobank/results/REGENIE_results/results_exome_data/070722_otosclerosis_200K/otosclerosis_200K_exome_regenie_2022-07-07.sbatch
INFO: Workflow csg (ID=w3eaa66dfb95cd32b) is executed successfully with 1 completed step.


## Rare-variant aggregate test for otosclerosis

In the burden files Isabelle asked to add start-loss to the LoF category

### Create the new annotation file

In [ ]:
tpl_file=$USER_PATH/bioworkflows/admin/csg.yml
annovar_dir=$UKBB_PATH/results/ukb23155_200Kexomes_annovar/070722_hg38_exome
annovar_sos=$USER_PATH/bioworkflows/variant-annotation/annovar.ipynb
annovar_sbatch=$UKBB_PATH/results/ukb23155_200Kexomes_annovar/070722_hg38_exome/ukb23155_200kexome_anno_startloss_$(date +"%Y-%m-%d").sbatch
bfiles=$UKBB_PATH/results/ukb23155_200Kexomes_annovar/091321_exome_bim_merge/ukb23155_chr1_chr22_091321.bim
walltime="60h"
mem="30G"

annovar_args="""annovar
    --cwd $annovar_dir 
    --bim_name $bfiles 
    --humandb /mnt/mfs/statgen/isabelle/REF/humandb  
    --xref_path /mnt/mfs/statgen/isabelle/REF/humandb 
    --job_size 1 
    --build 'hg38' 
    --name_prefix ukb23155_chr1_chr22_qcexomedata_070722
    --walltime $walltime
    --mem $mem
    --container_annovar /mnt/mfs/statgen/containers/gatk4-annovar.sif
"""

sos run ~/working/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $annovar_sos \
    --to-script $annovar_sbatch \
    --args "$annovar_args"